In [171]:
import librosa
import numpy as np
import tqdm

In [172]:
def griffinlim(spectrogram,
               n_iter = 100,
               window = 'hann',
               n_fft = 2048,
               hop_length = -1,
               verbose = False):
        """Implements the Griffin-Lim algorithm for phase
            reconstruction. Code found at
            https://github.com/librosa/librosa/issues/434. 

        @param spectrogram magnitudes of fft compoents by time step
        @param n_iter number of iteration of the algorithm
        @param window window function to use for overlapping samples
        @param n_fft number of Fourier components in spectrogram
                   @todo n_fft can in fact be extraced from spectrogram.
        @param hop_length distance between midpoint of overlapping
        time bins for fft. 
        @param verbose display progress info or not.
        @return stft with phases restored.
        """

        if hop_length == -1:
            # same default as for the librosa's [i]sftf function.
            hop_length = n_fft // 4

        # start with random phases/angles
        angles = np.exp(2j * np.pi * np.random.rand(*spectrogram.shape))
        print(spectrogram.shape)
        print(angles.shape)
        #print("______________")
        #print(np.abs(spectrogram).astype(np.complex))

        # use progress bar for iteration -- a gimmick
        t = tqdm.tqdm(range(n_iter), ncols=100, mininterval=2.0, disable=not verbose) 
        print(t)

        for i in t:

            full = np.dot(np.abs(spectrogram).astype(np.complex),angles)  #abs of spek(magnitudes) * random magnitudes
            
            inverse = librosa.istft(full, hop_length = hop_length, window = window) 
            rebuilt = librosa.stft(inverse, n_fft = n_fft,
            hop_length = hop_length, window = window) 
            angles = np.exp(1j * np.angle(rebuilt))

            if verbose:
                diff = np.abs(spectrogram) - np.abs(rebuilt)
                t.set_postfix(loss=np.linalg.norm(diff, 'fro'))

        return np.abs(spectrogram).astype(np.complex) * angles

In [193]:
audioPath = "../../gan/rnn/stft/stftreconstructed.wav"
#audioPath = "soundFiles/Test/test1.m4a"
    
audioData, sr = librosa.load(audioPath, sr=11025)
audioData = audioData[0:11025]    
 
print(len(audioData)/sr)

0.9752380952380952


In [194]:
stftData = librosa.stft(audioData)
absStftData = abs(stftData)
#print(stftData)
#print("_________")
print(absStftData)

[[0.07239701 0.0720422  0.12953109 ... 0.03051856 0.03845296 0.11870991]
 [0.05582868 0.00613326 0.11987438 ... 0.02761455 0.08153053 0.12875462]
 [0.06451701 0.02890061 0.1285662  ... 0.04365517 0.06108122 0.16167316]
 ...
 [0.7764682  0.4380449  0.7162789  ... 0.07170157 0.34409127 0.52925426]
 [0.80427766 0.50264126 0.84306145 ... 0.24207065 0.02755863 0.30685535]
 [0.86229247 0.356558   0.83569837 ... 0.27969074 0.30903253 0.3295638 ]]


In [195]:
reconstructed = librosa.griffinlim(absStftData)
print(reconstructed)

[ 0.01765062 -0.00769939 -0.04611916 ... -0.01044949  0.01976285
 -0.02607404]


In [196]:
import soundfile as sf

sf.write("reconstructedTest.wav", reconstructed, sr)